# 01 Introduction

Multilevel best linear unbiased estimators (MLBLUEs), orginially developed by Schaden and Ullmann [[paper](https://epubs.siam.org/doi/abs/10.1137/19M1263534)], are optimal multilevel/multifidelity estimators for the expectation of quantities of interest (QoIs)
affected by uncertainty. Given a set of models to use in the estimation, MLBLUEs automatically select the optimal selection and combination of models to use as well as the optimal number of samples required.
**BLUEST** implements the single- and multi-output MLBLUEs by Croci, Willcox and Wright [[paper](https://arxiv.org/pdf/2301.07831.pdf)], as well as the model selection and sample allocation optimal multilevel and multifidelity Monte Carlo methods (MLMC and MFMC respectively).

### Problem definition

In this tutorial we approximate $\mathbb{E}[e^Z]$, the expectation of $e^Z$, where $Z$ is a standard Gaussian random variable.

We use the following model set:
* We take the high-fidelity model to be $e^Z$.
* We define $3$ low-fidelity models by truncating the exponential series after $4$, $3$, and $2$ terms.
* We define the lowest-fidelity model to be $\log(|Z|)$ for no particular reason.

### Model set implementation

In terms of **BLUEST**, most of the users will only ever need to import the **BLUEST** problem class `BLUEProblem`:

In [1]:
from bluest import BLUEProblem

The `BLUEProblem` class must be overloaded to define the estimation problem. It is sufficient to only overload two class member functions: `sampler` and `evaluate`, with syntax:

**sampler**

The `sampler` function samples the model input random parameters.

The `sampler` function takes a list `ls` as input where `ls` is a list containing the indices of the models to be sampled. Note that the **highest-fidelity model must always correspond to the 0 index**.

The `sampler` function must return a list `samples` so that `len(samples) == len(ls)` so that `samples[i]` contains the input parameters for the model `ls[i]`. These input parameters must be either scalars or numpy arrays.

**IMPORTANT:** for the model samples to be correlated the inputs must also be statistically correlated. Often this simply means that the same outputs should be used for all models.

```python
def sampler(self, ls):
    # compute samples, a list of length len(ls)
    return samples

```

**evaluate**

The `evaluate` function takes the model input random parameters sampled with `sampler` and computes the model outputs.

The `evaluate` function takes `ls` and `samples` as input, where `ls` and `samples` are exactly the same as in the `sampler` function.

The `evaluate` function must return a list of list `out` with the following structure
```python
    L = len(ls)
    out = [[0 for i in range(L)] for n in range(n_outputs)] # n_outputs is an integer containing the number of output quantities of interest.
```

The syntax of `evaluate` for e.g., $1$ output is thus:

```python
    n_outputs = 1
    def evaluate(self, ls, samples):
        L = len(ls)
        out = [[0 for i in range(L)] for n in range(n_outputs)]

        for i in range(L):
            for n in range(n_outputs):
                model_number = ls[i]
                # recall that samples[ls[i]] contains the input parameters for model ls[i]
                # out[n][i] will thus contain the n-th output of the model model_number given the input samples[model_number]
                out[n][i] = ... # call model model_number with input samples[model_number]
                
        return out
```

We now go back to our problem. In this case the `BLUEProblem` class is overloaded as follows:

In [2]:
import numpy as np

# 5 models
n_models = 5

# define a function for the truncated exponential series
from scipy.special import gamma
def exponential_series(x,i):
    ii = np.arange(i+1)
    return np.sum(x**ii/gamma(ii+1)) # Euler Gamma function (recall \Gamma(n+1) = n!

# overload BLUEProblem and create a new class, MyProblem.
class MyProblem(BLUEProblem):
    def sampler(self, ls):
        L = len(ls)
        Z = np.random.randn()
        # IMPORTANT: float(Z) here is used to make copies of the same input, one for each model to be sampled.
        # If copies are not made, Python will only pass references to the same object which may lead to subtle bugs later on
        # in case the models modify the inputs in-place.
        samples = [float(Z) for i in range(L)]
        return samples

    def evaluate(self, ls, samples):
        L = len(ls)
        out = [[0 for i in range(L)]] # only one output, output 0

        for i in range(L):
            if ls[i] == 0:
                out[0][i] = np.exp(samples[i]) # high-fidelity model corresponding to model index 0
            elif ls[i] < n_models-1:
                out[0][i] = exponential_series(samples[i], n_models-ls[i]) # truncated exponential series
            else:
                out[0][i] = np.log(abs(samples[i])) # log(|Z|)

        return out


We can now create a `MyProblem` class object to setup the estimation problem:

In [3]:
# define costs somewhat arbitrarily. If not provided, they will
# be estimated via CPU time, which for this problem makes little sense.
costs = np.array([2**(n_models-i) for i in range(n_models)])

# Default verbose option is True for debugging, you can see everything that goes on
# under the hood if you set it to True. Advised if something breaks!
# 32 (or even 20) samples are typically enough for application runs. For debugging and Maths papers, set it to 1000.
# These samples won't be re-used. Sample re-use introduces bias and is not implemented here yet.
problem = MyProblem(n_models, costs=costs, covariance_estimation_samples=32, verbose=False)

The above code will instantiate a `problem` object of class `MyProblem`. **BLUEST** will assume by default that we are solving a single-output problem.

Instantiating `problem` will trigger an initial estimation of the covariance between all models using $32$ samples. If `costs` vector hadn't been provided, it would have taken two
extra samples to estimate the costs via CPU time.

### Part 1 - Basic Usage

We can ask **BLUEST** for the model covariance and correlation matrices, as well as the cost vector: 

In [4]:
print("Covariance matrix:\n")
print(problem.get_covariance())
print("\nCorrelation matrix:\n")
print(problem.get_correlation())
print("\nCost vector:\n")
print(problem.get_costs())

Covariance matrix:

[[2.53500581 2.3583669  2.36312599 1.66331444 0.72897923]
 [2.3583669  2.27345322 2.07702441 1.67029559 0.80520217]
 [2.36312599 2.07702441 2.40127866 1.37242096 0.47072499]
 [1.66331444 1.67029559 1.37242096 1.29027193 0.67275196]
 [0.72897923 0.80520217 0.47072499 0.67275196 1.56637342]]

Correlation matrix:

[[1.         0.98237857 0.95780283 0.9196959  0.36582891]
 [0.98237857 1.         0.88894995 0.97523594 0.42669178]
 [0.95780283 0.88894995 1.         0.77969623 0.24271595]
 [0.9196959  0.97523594 0.77969623 1.         0.47322393]
 [0.36582891 0.42669178 0.24271595 0.47322393 1.        ]]

Cost vector:

[32 16  8  4  2]


Let's start solving the estimatione problem. For this purpose, we need to prescribe a statistical error tolerance $\varepsilon$ or a computational budget.

Let's start by prescribing a tolerance of $0.01$ times the standard deviation of the high-fidelity model:

In [5]:
# define statistical error tolerance
eps = 0.01*np.sqrt(problem.get_covariance()[0,0])

**BLUEST** offers $4$ different estimators: standard Monte Carlo, MLMC, MFMC, and MLBLUE.

To use standard Monte Carlo, we can call

In [6]:
# solve with standard MC
sol_MC = problem.solve_mc(eps=eps)
print("\n\nStd MC\n")
print("Std MC solution: ", sol_MC[0], "\nTotal cost: ", sol_MC[2]) # sol_MC[1] here contains the statistical error, which in this case should be <= eps



Std MC

Std MC solution:  [1.641081294514136] 
Total cost:  320000


The above will compute the samples and return the estimator. Of course, standard Monte Carlo is the slowest method and **BLUEST** can do better.

A feature of **BLUEST** is that we can inquire about the cost of MLMC, MFMC and MLBLUE **without actually running any samples**!

In [7]:
# MLMC
print("\n\nMLMC\n")
MLMC_data = problem.setup_mlmc(eps=eps)
for key, item in MLMC_data.items(): print(key, ": ", item)

# MFMC
print("\n\nMFMC\n")
MFMC_data = problem.setup_mfmc(eps=eps)
for key, item in MFMC_data.items(): print(key, ": ", item)

# MLBLUE. K denotes the maximum model group size allowed, see our paper.
print("\n\nMLBLUE\n")
MLBLUE_data = problem.setup_solver(K=n_models, eps=eps)
for key, item in MLBLUE_data.items(): print(key, ": ", item)



MLMC

models :  [0, 1, 3]
samples :  [ 1118  2701 14523]
errors :  [0.015921651552570866]
total_cost :  137084


MFMC

models :  [0, 3]
samples :  [ 2819 18671]
errors :  [0.01592076469695972]
total_cost :  164892
alphas :  [array([1.28911929])]


MLBLUE

models :  [[2], [3], [0, 2, 3], [1, 2, 3], [0, 1, 2, 3]]
samples :  [5938 6797    1  493   43]
errors :  [0.01592249]
total_cost :  91120


Note that here the outputs have slightly different meanings depending on the methods.
We recommend the reader to familiarise themselves with the MLBLUE, MLMC and MFMC methods to have a clearer picture.

For MLMC and MFMC, the variable models contains a list of the models used, while for MLBLUE it contains a list of the model groupings used. **Note** that **BLUEST** has automatically discarded some models as their use has been found suboptimal. The final model choice is of course method-dependent.

For MLMC, `samples[i]` contains the number of samples that MLMC would take on MLMC level i. Recall that MLMC on a given level computes samples of differences between models, so that `samples[0]` contains the number of samples taken of the difference between model number `models[0]` and model number `models[1]`.

For MFMC, `samples[i]` simply contains the number of samples taken from model number `models[i]`. The variable `alpha` contains the optimal MFMC control-variate coefficients.

For MLBLUE, `samples[i]` simply contains the number of samples taken from all models in the model group `models[i]`.

---------------------------------------------------------------------------------------------------------------------------

Generally speaking, we recommend that the user should always check the costs of the methods since it is very quick and often very informative. Note that MLBLUE should always be more efficient (or more accurate) than the other methods. However, the setup of MLBLUE requires nonlinear integer optimization and, while unlikely, it may happen that the integer solution found is sub-optimal (see paper).

To actually use the methods and start sampling, we can simply call:

In [8]:
# Solve with MLMC
print("\n\nMLMC\n")
sol_MLMC = problem.solve_mlmc(eps=eps)
print("MLMC solution: ", sol_MLMC[0])

# Solve with MFMC
print("\n\nMFMC\n")
sol_MFMC = problem.solve_mfmc(eps=eps)
print("MFMC solution: ", sol_MFMC[0])

# Solve with MLBLUE. K denotes the maximum group size allowed.
print("\n\nMLBLUE\n")
sol_MLBLUE = problem.solve(K=n_models, eps=eps)
print("MLBLUE solution: ", sol_MLBLUE[0])

# MLBLUE is more sensitive than the other methods to integer projection,
# always good to check all methods. This does not require any sampling.
MLMC_data   = problem.setup_mlmc(eps=eps)
MFMC_data   = problem.setup_mfmc(eps=eps)
MLBLUE_data = problem.setup_solver(K=n_models, eps=eps)
print("\nCost comparison. MLMC: %f, MFMC: %f, MLBLUE: %f" % (MLMC_data["total_cost"], MFMC_data["total_cost"], MLBLUE_data["total_cost"]))



MLMC

MLMC solution:  [1.6462603578070634]


MFMC

MFMC solution:  [1.6367389524750158]


MLBLUE

MLBLUE solution:  [1.6328484153177762]

Cost comparison. MLMC: 137084.000000, MFMC: 164892.000000, MLBLUE: 91120.000000


Note that calling solve will internally call the methods' setup routines as well. For MLBLUE only, previous setup calls will be saved into the `BLUEProblem` object for speed.
This is especially useful when the user wants to try out different MLBLUE setup strategies, e.g., rather than prescribing the maximum model group size, we can just prescribe the allowed
model groupings directly (this will of course affect the total estimation cost):

In [9]:
# Alternatively, can specify which groups to use:
print("\n\nMLBLUE\n")
groups = [[0], [1], [0,3], [4,5], [0,1,2,3,4]]
MLBLUE_data = problem.setup_solver(groups=groups, eps=eps)
print("MLBLUE data:\n")
for key, item in MLBLUE_data.items(): print(key, ": ", item)



MLBLUE

MLBLUE data:

models :  [[1], [0, 3], [0, 1, 2, 3, 4]]
samples :  [9882 1794  305]
errors :  [0.01592225]
total_cost :  241606


It is also possible to prescribe a budget:

In [10]:
# You can also set a budget rather than a tolerance, e.g.
budget = 100*max(costs) # budget corresponding to 100 std MC samples
# same syntax for MLMC and MFMC. Never provide both eps and budget at the same time
MLBLUE_data = problem.setup_solver(K=n_models, budget=budget)

#NOTE: no need for calling setup_solver, can call solve directly, although we do not recommend it
#      as it is always better to check first.

**IMPORTANT** MLBLUE optimization solver. Most of the time this is not needed, but it may happen that the MLBLUE optimization solver fails due to a lack of feature/bug in CVXPY/CVXOPT for which the optimizer does not recognise it has found a solution, and it keeps iterating until failure. If this happens, you can tweak the optimization solver parameters as follows:

In [11]:
cvxopt_params = {
        "abstol" : 1.e-7,
        "reltol" : 1.e-4,
        "maxiters" : 1000, # called max_iters for "cvxpy"
        "feastol" : 1.0e-6,
}
MLBLUE_data = problem.setup_solver(K=n_models, budget=budget, solver="cvxopt", optimization_solver_params=cvxopt_params)
# Changing feastol is typically enough, sometimes you can increase the other tolerances or reduce maxiters

### Part 2 - Parallelization

**BLUEST** runs in parallel with MPI. Simply call the script with e.g.:

```bash
> mpiexec -n NPROCS python3 minimal.py 
```
And all the sampling will be split across `NPROCS` workers and occur in parallel (N.B. number of pilot samples will be
rounded up so that it is a multiple of `NPROCS`, but the number of online samples will not be increased).

#On computing nodes it is often best to set the `OMP_NUM_THREADS` flag to avoid unwanted multithreading, e.g.,

```bash
> OMP_NUM_THREADS=1 mpiexec -n NPROCS python3 minimal.py 
```

**NOTE:** BLUEST uses mpi4py for MPI parallelization.

**NOTE:** Always make sure that the random number generator you use is thread-safe and that you are using independent generators for each worker by using skipahead functionalities (if implemented) or different random seeds. e.g. in Python:

In [12]:
from mpi4py import MPI
from numpy.random import RandomState

comm = MPI.COMM_WORLD
mpiRank = comm.Get_rank()
mpiSize = comm.Get_size()

RNG = RandomState(mpiRank) # sets a different seed for each worker's random number generator
RNG.randn()

1.764052345967664

BLUEST will handle the rest of the parallelization for you so that you do not have to worry about it apart from the random number generator RNG.

**NOTE:** If your models also use MPI, then you need to be careful. Simplest option is to set the MPI communicator of your models to `MPI_COMM_SELF` so that each worker loads the full model and deadlocks are avoided. However, BLUEST does support nested MPI communicators: the `BLUEProblem` takes an mpi4py communicator as input:

```python
    # here mycomm is an mpi4py communicator
    problem = MyProblem(n_models, costs=costs, comm = mycomm, covariance_estimation_samples=32, verbose=False)
```

In the above, samples will be parallelised across each rank of `mycomm` and it is up to the user to modify the `BLUEProblem` class accordingly so that each rank does the right thing and passes the correct MPI communicator to the models. E.g., if we use $4$ workers split into two pairs, we can create a subcommunicator for each pair and a cross-communicator between the two pairs. In this case, we need to pass the cross-communicator to `BLUEProblem` and use the subcommunicators as the MPI communicators of the models so that samples are parallelised across pairs and each model sample is parallelised using a single pair.

For an (alas, contrived) example on the use of nested MPI communicators, see the restrictions_matern example in the example/paper_example folder in the **BLUEST** repository. The developers are available via email for any questions.

### Part 3 - Advanced usage

In [13]:
# First some cleanup
# Nothing BLUEST-specific in the next two lines: just creating/cleaning up a temporary directory for this script
import shutil; shutil.rmtree("/tmp/mlblue/", ignore_errors=True) # cleaning up from previous runs
import os; os.makedirs("/tmp/mlblue",exist_ok=True) # creating temporary directory for the script

Since the offline covariance estimation can be costly, **BLUEST** allows the user to save offline estimation data and load them later, or to use user-provided covariances and cost vectors. 

In [14]:
# Can save offline estimation data and load them later.
problem.save_graph_data("/tmp/mlblue/minimal_data.npz")
problem = MyProblem(n_models, datafile="/tmp/mlblue/minimal_data.npz")

# Can also overload cost vector when loading.
problem = MyProblem(n_models, costs=costs, datafile="/tmp/mlblue/minimal_data.npz")

# Similarly, you can avoid offline sampling if covariance and costs are known.
C = np.random.randn(n_models, n_models); C = C.T@C;
problem = MyProblem(n_models, C = C, costs=costs, verbose=False)

# ONLY FOR MLMC:
# For MLMC only, the variance of model differences V[P_i-P_j] must be also provided.
# In this case only an upper triangular block is needed,
# with dV[i,j] = V[P_i - P_j]. The rest must be set to NaN
dV = np.nan*np.ones_like(C)
for i in range(n_models):
    for j in range(i+1,n_models):
        # V[P_i - P_j] = V[P_i] + V[P_j] - 2*C(P_i, P_j)
        dV[i,j] = C[i,i] + C[j,j] - 2*C[i,j]

problem = MyProblem(n_models, C = C, mlmc_variances=dV, costs=costs, verbose=False)
# When estimating the covariance with pilot samples, the MLMC variances will also be estimated
# and they will also be saved to file with save_graph_data and loaded automatically afterwards.
# you can access them with
problem.get_mlmc_variance()


BLUE estimator ready.


BLUE estimator ready.



array([[        nan,  3.06620309, 11.97634042,  7.66229986,  6.77110948],
       [        nan,         nan,  7.70089361,  6.4406335 ,  2.65369473],
       [        nan,         nan,         nan, 12.14401703,  6.38489099],
       [        nan,         nan,         nan,         nan,  2.58219465],
       [        nan,         nan,         nan,         nan,         nan]])

It is also possible to re-estimate some covariances or to exclude some model couplings:

In [15]:
# Can ask to re-estimate some entries by setting entries of C to NaN. Note that
# all models might need to be sampled so you might as well re-estimate everything
C[0,0] = np.nan
problem = MyProblem(n_models, C = C, costs=costs, covariance_estimation_samples=32, verbose=False)

# Can exclude model groups also by setting covariance entries to inf, e.g.
C = np.nan*C # setting all entries of C to NaN, they will be re-estimated
# The covariance between Model 0 and Model 1 won't be estimated
# and the two models will never be sampled together
# Model set might be pruned after this in case some models become useless
C[0,1] = np.inf; C[1,0] = np.inf
problem = MyProblem(n_models, C = C, costs=costs, covariance_estimation_samples=32)

Covariance estimation with 32 samples...
Sampling of models [0, 1, 2, 3, 4] completed.                                                                                                                                                                      
Running Spectral Gradient Descent for Covariance projection...
Covariance projected, projection error:  3.5252221702063965e-30

BLUE estimator ready.



Note that independently from whether the covariance matrices are estimated or user-prescribed, they will be checked for positive definiteness. If they are not positive definite,
they will be projected to be positive definite. You can skip this projection at your own risk by setting `skip_projection=True`.

In [16]:
# You can skip this projection at your own risk by setting skip_projection=True
problem = MyProblem(n_models, C = C, costs=costs, covariance_estimation_samples=32, skip_projection=True)

Covariance estimation with 32 samples...
Sampling of models [0, 1, 2, 3, 4] completed.                                                                                                                                                                      

BLUE estimator ready.



Note that the MLBLUE method requires the covariance matrix to be positive definite so the best course of action may be to actually remove a model by hand rather than skipping or using the projection.
This is one of the cases in which you do want to check whether MLMC of MFMC do better than MLBLUE: MLMC and MFMC are unaffected by singular covariances. See our paper for further details about singular covariances.

In [17]:
# NOT SO IMPORTANT:
# You can tweak the parameters for the spd projection, but it is almost never needed
spg_params = {"maxit" : 10000,
              "maxfc" : 10000**2,
              "verbose" : False,
              "spd_threshold" : 5.0e-14,  # minimum eigenvalue
              "eps"     : 1.0e-10,        # optimization solver tolerance
              "lmbda_min" : 10.**-30,
              "lmbda_max" : 10.**30,
              "linesearch_history_length" : 10,
             }
# In some very rare cases with near-singular covariance matrices spd_threshold and eps might require some tweaking. 
problem = MyProblem(n_models, C = C, costs=costs, covariance_estimation_samples=32, spg_params=spg_params)

# The projection uses simple SVD hard thresholding for covariances without excluded model groups. Otherwise BLUEST uses a spectral gradient descent method.

Covariance estimation with 32 samples...
Sampling of models [0, 1, 2, 3, 4] completed.                                                                                                                                                                      
Running Spectral Gradient Descent for Covariance projection...
Covariance projected, projection error:  8.960196473267176e-30

BLUE estimator ready.



You can ask **BLUEST** to save all sample outputs (e.g. snapshots), and their input parameters. All samples will be saved in different npz files with naming convention `snapshots$MODELS.npz`
where `$MODELS` corresponds to which models are sample together. New samples will always be appended.

In [18]:
# You can ask BLUEST to save all sample outputs (e.g. snapshots), and their input parameters
C = np.random.randn(n_models, n_models); C = C.T@C;
problem = MyProblem(n_models, C = C, costs=costs, samplefile="/tmp/mlblue/snapshots.npz", verbose=False)

# all samples will be saved in different npz files with naming convention snapshots$MODELS.npz
# where $MODELS corresponds to which models are sample together. New samples will always be appended.
sol_MLBLUE = problem.solve(K=n_models, eps=eps)

# You can avoid saving the pilot samples by setting a samplefile later
problem = MyProblem(n_models, costs=costs, covariance_estimation_samples=32, verbose=False)
problem.params["samplefile"] = "/tmp/mlblue/snapshots.npz"

# You can change samplefile name as you go
problem.params["samplefile"] = "/tmp/mlblue/snapshots_MLMC.npz" # set specific filename for MLMC
sol_MLMC = problem.solve_mlmc(eps=eps)
problem.params["samplefile"] = "/tmp/mlblue/snapshots.npz" # reset to default

Batch sampling (taking multiple samples in one go) is supported, but untested. For batch sampling, it is sufficient to define the `sampler` and `evaluate` functions as follows:

```python
def sampler(self, ls, Nbatch=1):
    ...

def evaluate(self, ls, samples, Nbatch=1):
    # Here out[0][i][n] is the n_th sample in the batch for model i and output 0
    return out
```

If you are interested in this functionality please let the developers know, especially if you find bugs.

## Part 4 - Multiple outputs

Same problem as before, but now we consider two outputs: $e^Z$ and $e^{2Z}$. We use the same models at before to approximate $e^Z$ (and we square the result to approximate $e^{2Z}$).

We define the multi-output `BLUEProblem` as follows:

In [19]:
# Now consider two outputs: e^Z and (e^Z)^2
n_outputs = 2

class MyMultiOutputProblem(BLUEProblem):
    def sampler(self, ls):
        L = len(ls)
        Z = RNG.randn()
        samples = [float(Z) for i in range(L)]
        return samples

    def evaluate(self, ls, samples):
        L = len(ls)
        #NOTE the definition of the output.
        out = [[0 for i in range(L)] for n in range(n_outputs)]

        pw = [1,2]
        for i in range(L):
            for n in range(n_outputs):
                #NOTE the indexing of out
                if ls[i] == 0:
                    out[n][i] = np.exp(samples[i])**pw[n]
                elif ls[i] < n_models-1:
                    out[n][i] = exponential_series(samples[i], n_models-ls[i])**pw[n]
                else:
                    # let's reuse the same quantity for both outputs, why not?
                    out[n][i] = np.log(abs(samples[i]))

        return out

# NOTE: costs are the same as before, i.e. we assume that each model evaluation costs the same for all QoI that it outputs.
#       different costs vectors for each output currently not supported. Raise an issue if needed.
costs = np.array([2**(n_models-i) for i in range(n_models)])

# Now need to specify how many outputs when instantiating the problem object
problem = MyMultiOutputProblem(n_models, n_outputs=n_outputs, costs=costs, covariance_estimation_samples=32, verbose=False)

Note that we now had to specify how many outputs we have to the overloaded `BLUEProblem` class instantiation. The rest is mainly the same as before.

We setup the multi-output estimators as follows:

In [20]:
# same as before with prescribed budget
budget = 1000*max(costs) # budget corresponding to 1000 std MC samples
MLBLUE_data = problem.setup_solver(K=n_models, budget=budget)

# can prescribe a single statistical error tolerance for all outputs:
eps = 0.01*np.sqrt(problem.get_covariance(0)[0,0])
MLBLUE_data = problem.setup_solver(K=n_models, eps=eps)
# or a different tolerance for different outputs
eps = [0.01*np.sqrt(problem.get_covariance(n)[0,0]) for n in range(n_outputs)]
MLBLUE_data = problem.setup_solver(K=n_models, eps=eps)

# Can prescribe a single group for all
groups = [[0], [1], [0,3], [4,5], [0,1,2,3,4]]
MLBLUE_data = problem.setup_solver(groups=groups, eps=eps)

# Or different groups for each
groups_0 = [[0], [1], [0,3], [4,5], [0,1,2,3,4]]
groups_1 = [[0], [1], [1,3], [3,5], [0,1,3,4]]
multi_groups = [groups_0, groups_1]
#NOTE: the follwing is untested, if it breaks please raise an issue
MLBLUE_data = problem.setup_solver(multi_groups=multi_groups, eps=eps)

We can still prescribe and/or request the model covariances as follows:

In [21]:
# now need to prescribe a model covariance (and mlmc_variances) for each QoI:
# (can use nan and inf as before)
C0 = np.random.randn(n_models, n_models); C0 = C0.T@C0;
C1 = np.random.randn(n_models, n_models); C1 = C1.T@C1;
C = [C0, C1]
dV = [np.nan*np.ones_like(c) for c in C]
for n in range(n_outputs):
    for i in range(n_models):
        for j in range(i+1,n_models):
            dV[n][i,j] = C[n][i,i] + C[n][j,j] - 2*C[n][i,j]

# each covariance matrix will be projected to be spd
problem = MyMultiOutputProblem(n_models, n_outputs=n_outputs, C = C, mlmc_variances=dV, costs=costs, verbose=False)

# can get them all
C = problem.get_covariances()
R = problem.get_correlations()
dV = problem.get_mlmc_variances()
print(C)
print(R)
print(dV)

# or one at a time (the indices 0 and 1 correspond to outputs 0 and 1 respectively)
C0 = problem.get_covariance(0)
R0 = problem.get_correlation(0)
dV0 = problem.get_mlmc_variance(0)
C1 = problem.get_covariance(1)
R1 = problem.get_correlation(1)
dV1 = problem.get_mlmc_variance(1)

print(C0)
print(R0)
print(dV0)

[array([[ 4.65733401,  3.64808022, -0.05067521, -2.72783366, -0.54276508],
       [ 3.64808022,  4.96631701,  0.03092927, -2.8008079 , -0.89574066],
       [-0.05067521,  0.03092927,  2.52874353,  0.51137754, -0.37679739],
       [-2.72783366, -2.8008079 ,  0.51137754,  3.74978472, -1.28059895],
       [-0.54276508, -0.89574066, -0.37679739, -1.28059895,  2.19306546]]), array([[ 3.94709133,  1.63920466,  2.40555303,  0.85065015,  1.22081507],
       [ 1.63920466,  2.08853369,  0.42852247, -0.02972595, -1.72286796],
       [ 2.40555303,  0.42852247,  2.61809801,  0.85195781,  1.466295  ],
       [ 0.85065015, -0.02972595,  0.85195781,  0.90699405,  0.37464459],
       [ 1.22081507, -1.72286796,  1.466295  ,  0.37464459,  4.86135647]])]
[array([[ 1.        ,  0.75853997, -0.01476639, -0.65274871, -0.16983117],
       [ 0.75853997,  1.        ,  0.00872771, -0.64902726, -0.27141839],
       [-0.01476639,  0.00872771,  1.        ,  0.16606805, -0.16000358],
       [-0.65274871, -0.64902726

We can still store samples for all outputs (or just for some outputs) to file:

In [22]:
# Can still store all the samples for all the outputs to file
problem = MyMultiOutputProblem(n_models, n_outputs=n_outputs, C = C, costs=costs, samplefile="/tmp/mlblue/snapshots_multi.npz", verbose=False)
# or only some of the outputs. Just put them in a list.
outputs_to_save = [1]
problem = MyMultiOutputProblem(n_models, n_outputs=n_outputs, C = C, costs=costs, outputs_to_save = outputs_to_save, samplefile="/tmp/mlblue/snapshots_multi_again.npz", verbose=False)

**NOTE:** everything else should be the same as for the single-output case. Any questions please ask the developers!

**NOTE:** full python script available in this folder `01_tutorial.py`.